In [16]:
# Імпортуємо необхідні бібліотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


iris = pd.read_csv('IRIS.csv')

X = iris.iloc[:, :-1]  # Ознаки (всі стовпці, крім останнього)
Y = iris.iloc[:, -1]   # Цільова змінна (останній стовпець)


# Розділення даних на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Ініціалізація моделі KNN
k = 3  # Кількість сусідів
knn = KNeighborsClassifier(n_neighbors=k)

# Навчання моделі на тренувальних даних
knn.fit(X_train, y_train)

# Прогнози на тестовому наборі
y_pred = knn.predict(X_test)

# Оцінка результатів класифікаці
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)



Accuracy: 1.0
Classification Report:
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        10
Iris-versicolor       1.00      1.00      1.00         9
 Iris-virginica       1.00      1.00      1.00        11

       accuracy                           1.00        30
      macro avg       1.00      1.00      1.00        30
   weighted avg       1.00      1.00      1.00        30

